# Get Congress Members

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import os
from itertools import groupby
import numpy as np

In [2]:
url2 = 'https://en.wikipedia.org/wiki/List_of_current_United_States_senators'
response2 = rq.get(url2)

soup2 = BeautifulSoup(response2.content,'html.parser')

tables = soup2.find_all('table')
party_state = pd.read_html(str(tables[5]))
party_state = pd.DataFrame(party_state[0])

split_names = party_state['Senator'].str.split(expand=True)
party_state['First_Name'] = split_names[0]

def get_last_name(row):
    if pd.isna(row[2]):
        return row[1]
    if row[2] == 'Jr.':
        return f'{row[1]}'
    else:
        return f'{row[2]}'

party_state['Last_Name'] = split_names.apply(get_last_name, axis=1)

# replace Bernie with Bernard
party_state.loc[(party_state['First_Name'] == 'Bernie') & (party_state['Last_Name'] == 'Sanders'), 'First_Name'] = 'Bernard'
party_state.loc[(party_state['First_Name'] == 'Ben') & (party_state['Last_Name'] == 'Cardin'), 'First_Name'] = 'Benjamin'
party_state.loc[(party_state['First_Name'] == 'Bob') & (party_state['Last_Name'] == 'Menendez'), 'First_Name'] = 'Robert'
party_state.loc[(party_state['First_Name'] == 'Bob') & (party_state['Last_Name'] == 'Casey'), 'First_Name'] = 'Robert'
party_state.loc[(party_state['First_Name'] == 'Tom') & (party_state['Last_Name'] == 'Carper'), 'First_Name'] = 'Thomas'
party_state.loc[(party_state['First_Name'] == 'Jim') & (party_state['Last_Name'] == 'Risch'), 'First_Name'] = 'James'
party_state.loc[(party_state['First_Name'] == 'Chris') & (party_state['Last_Name'] == 'Murphy'), 'First_Name'] = 'Christopher'
party_state.loc[(party_state['First_Name'] == 'Chris') & (party_state['Last_Name'] == 'Coons'), 'First_Name'] = 'Christopher'
party_state.loc[(party_state['First_Name'] == 'Dick') & (party_state['Last_Name'] == 'Durbin'), 'First_Name'] = 'Richard'
party_state.loc[(party_state['First_Name'] == 'Maggie') & (party_state['Last_Name'] == 'Hassan'), 'First_Name'] = 'Margaret'
party_state.loc[(party_state['First_Name'] == 'Ed') & (party_state['Last_Name'] == 'Markey'), 'First_Name'] = 'Edward'
party_state.loc[(party_state['First_Name'] == 'J.') & (party_state['Last_Name'] == 'Vance'), 'First_Name'] = 'J.D.'
party_state.loc[(party_state['First_Name'] == 'Chuck') & (party_state['Last_Name'] == 'Schumer'), 'First_Name'] = 'Charles'

# search for Sanders
# party_state[party_state['Last_Name'] == 'Vance']


/var/folders/fs/rvvymn3n2rs6sxttssj5wr3w0000gn/T/ipykernel_2292/672875004.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  party_state = pd.read_html(str(tables[5]))


## Get Senate Members

In [3]:
url2 = 'https://www.senate.gov/general/committee_assignments/assignments.htm'
response2 = rq.get(url2)

soup2 = BeautifulSoup(response2.content,'html.parser')

gov_data = soup2.select('a')

actual_titles = []
for title in gov_data:
    if title.parent.name == 'div' or title.parent.name == 'strong':
        actual_titles.append(title.text.strip())


In [4]:
arr = np.array(actual_titles)
idx = np.where(arr == 'Back to top')[0]
subarrays = np.split(arr, idx+1)

result = [subarray.tolist() for subarray in subarrays if len(subarray) > 0]

filtered_result = [[item for item in sublist if item != '' and item != 'Back to top'] for sublist in result]

senators_committees = pd.DataFrame(filtered_result).rename(columns={0:'senator',1:'committee_1',2:'committee_2',3:'committee_3',4:'committee_4',5:'committee_5',6:'committee_6',7:'committee_7',8:'committee_8'})

In [5]:
senators_committees

,senator,committee_1,committee_2,committee_3,committee_4,committee_5,committee_6,committee_7,committee_8
0,"Baldwin, Tammy",Committee on Appropriations,"Committee on Commerce, Science, and Transporta...","Committee on Health, Education, Labor, and Pen...",None,None,None,None,None
1,"Barrasso, John",Committee on Energy and Natural Resources,Committee on Finance,Committee on Foreign Relations,None,None,None,None,None
2,"Bennet, Michael F.","Committee on Agriculture, Nutrition, and Forestry",Committee on Finance,Committee on Rules and Administration,Select Committee on Intelligence,None,None,None,None
3,"Blackburn, Marsha","Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on the Judiciary,Committee on Veterans' Affairs,None,None,None,None
4,"Blumenthal, Richard",Commission on Security and Cooperation in Europe,Committee on Armed Services,Committee on Homeland Security and Governmenta...,Committee on the Judiciary,Committee on Veterans' Affairs,Special Committee on Aging,United States Senate Caucus on International N...,None
...,...,...,...,...,...,...,...,...,...
96,"Whitehouse, Sheldon",Commission on Security and Cooperation in Europe,Committee on Environment and Public Works,Committee on Finance,Committee on the Budget,Committee on the Judiciary,United States Senate Caucus on International N...,None,None
97,"Wicker, Roger F.",Commission on Security and Cooperation in Europe,Committee on Armed Services,"Committee on Commerce, Science, and Transporta...",Committee on Environment and Public Works,Committee on Rules and Administration,Select Committee on Intelligence,None,None
98,"Wyden, Ron",Committee on Energy and Natural Resources,Committee on Finance,Committee on the Budget,Joint Committee on Taxation,Select Committee on Intelligence,None,None,None
99,"Young, Todd","Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on Foreign Relations,Committee on Small Business and Entrepreneurship,None,None,None,None


In [6]:
senators_committees['first_name'] = senators_committees['senator'].str.split(', ').str[1]
senators_committees['last_name'] = senators_committees['senator'].str.split(', ').str[0]
senators_committees['full_name'] = senators_committees['first_name'] + ' ' + senators_committees['last_name']

In [7]:
senators_committees['first_name'] = senators_committees['first_name'].str.split(' ').str[0]
senators_committees['last_name'] = senators_committees['last_name'].str.split(' ').str[-1]

In [8]:
senators_committees[5:]

,senator,committee_1,committee_2,committee_3,committee_4,committee_5,committee_6,committee_7,committee_8,first_name,last_name,full_name
5,"Booker, Cory A.","Committee on Agriculture, Nutrition, and Forestry",Committee on Foreign Relations,Committee on Small Business and Entrepreneurship,Committee on the Judiciary,None,None,None,None,Cory,Booker,Cory A. Booker
6,"Boozman, John",Commission on Security and Cooperation in Europe,"Committee on Agriculture, Nutrition, and Forestry",Committee on Appropriations,Committee on Environment and Public Works,Committee on Veterans' Affairs,None,None,None,John,Boozman,John Boozman
7,"Braun, Mike","Committee on Agriculture, Nutrition, and Forestry","Committee on Health, Education, Labor, and Pen...",Committee on the Budget,Special Committee on Aging,None,None,None,None,Mike,Braun,Mike Braun
8,"Britt, Katie Boyd",Committee on Appropriations,"Committee on Banking, Housing, and Urban Affairs",Committee on Rules and Administration,None,None,None,None,None,Katie,Britt,Katie Boyd Britt
9,"Brown, Sherrod","Committee on Agriculture, Nutrition, and Forestry","Committee on Banking, Housing, and Urban Affairs",Committee on Finance,Committee on Veterans' Affairs,None,None,None,None,Sherrod,Brown,Sherrod Brown
...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Whitehouse, Sheldon",Commission on Security and Cooperation in Europe,Committee on Environment and Public Works,Committee on Finance,Committee on the Budget,Committee on the Judiciary,United States Senate Caucus on International N...,None,None,Sheldon,Whitehouse,Sheldon Whitehouse
97,"Wicker, Roger F.",Commission on Security and Cooperation in Europe,Committee on Armed Services,"Committee on Commerce, Science, and Transporta...",Committee on Environment and Public Works,Committee on Rules and Administration,Select Committee on Intelligence,None,None,Roger,Wicker,Roger F. Wicker
98,"Wyden, Ron",Committee on Energy and Natural Resources,Committee on Finance,Committee on the Budget,Joint Committee on Taxation,Select Committee on Intelligence,None,None,None,Ron,Wyden,Ron Wyden
99,"Young, Todd","Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on Foreign Relations,Committee on Small Business and Entrepreneurship,None,None,None,None,Todd,Young,Todd Young


In [9]:
senators_committees_merged = senators_committees.merge(party_state, left_on=['last_name','first_name'], right_on=['Last_Name','First_Name'], validate='1:1', how='left')
senators_committees_merged = senators_committees_merged[['senator', 'first_name', 'last_name', 'full_name', 'committee_1','committee_2','committee_3','committee_4','committee_5','committee_6','committee_7','committee_8','Party.1','State']]

In [10]:
senators_committees_merged = senators_committees_merged[:100]


In [11]:
DATA_PATH = '../../data'

if not os.path.exists(DATA_PATH+'/inputs'):
    os.makedirs(DATA_PATH+'/inputs')

senators_committees_csv = '/inputs/senators_committees.csv'

if not os.path.exists(senators_committees_csv):
    senators_committees.to_csv(DATA_PATH+senators_committees_csv,index=False)

## Get gsectors

In [12]:
# url3 = 'https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard'
# response3 = rq.get(url3)

# soup3 = BeautifulSoup(response3.content,'html.parser')

# gsect = soup3.select('table')
# gsect = pd.read_html(str(gsect))
# gsect = pd.DataFrame(gsect[0])

In [13]:
# gsect